In [1]:
# open /home/as2114/code/3DBB/data/dl_challenge/8b061a8a-9915-11ee-9103-bbb8eae05561/pc.npy
import numpy as np
pc = np.load('/home/as2114/code/3DBB/data/dl_challenge/8b061a8a-9915-11ee-9103-bbb8eae05561/pc.npy')

In [2]:
pc.shape

(3, 481, 607)